# Base year (2023) validation

This notebook validates the results of PyPSA-Earth for the base year (2023) against the **EIA (U.S. Energy Information Administration)** and **Ember** US datasets. The validation covers three key aspects of the electricity system:
- **Electricity Demand**
- **Electricity Generation**
- **Installed Generation Capacity**

## 1. Setup and Data Loading

*This section handles the initial setup, including importing necessary libraries and loading the solved PyPSA-Earth networks.*

### 11.1 Loading libraries

We begin by importing the necessary libraries for data handling, analysis, and visualization. These include PyPSA for power system analysis, pandas and numpy for data manipulation, geopandas for spatial data, and seaborn/matplotlib for plotting.

---

In [1]:
# Install required packages if not already installed
# Uncomment the line below to install packages
# Note: This line is commented out to prevent installation during code execution.

# !pip install numpy pandas plotly pycountry matplotlib seaborn -qq

In [2]:
import os
import pypsa
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

import pycountry

from _helpers import *

import warnings
warnings.filterwarnings("ignore")

/home/user/miniforge3/envs/pypsa-earth/lib/python3.10/site-packages/pypsa/networkclustering.py:16: UserWarning:

The namespace `pypsa.networkclustering` is deprecated and will be removed in PyPSA v0.24. Please use `pypsa.clustering.spatial instead`. 



In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 
pd.set_option('display.width', None)    
pd.set_option('display.max_colwidth', None)

### 1.3 Loading Files

PyPSA-Earth solved networks for the base year are uploaded here, as well as the relevant EIA and Ember datasets. These datasets provide reference values for demand, generation, and installed capacity, which will be used for validation.

In [4]:
DATA_DIR = "../notebooks/validation_data/"

ember_data_path = os.path.join(DATA_DIR, "ember_yearly_full_release_long_format.csv")
eia_demand_path = os.path.join(DATA_DIR, "EIA_demand.xlsx")
eia_installed_capacities_path = os.path.join(DATA_DIR, "EIA_installed_capacities.csv")
eia_generation_path = os.path.join(DATA_DIR, "EIA_electricity_generation.csv")

In [5]:
ember_data = pd.read_csv(ember_data_path)

In [7]:
# load network
project_root = os.getcwd()
results_dir = os.path.join(project_root, 'results')

# Load Base year network
base_path = os.path.join(results_dir, 'base_year', "elec_s_100_ec_lcopt_Co2L-3H_3H_2020_0.07_AB_0export.nc")
base_network = pypsa.Network(base_path)

INFO:pypsa.io:Imported network elec_s_100_ec_lcopt_Co2L-3H_3H_2020_0.07_AB_0export.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


In [8]:
state_shape_path = "gadm41_USA_1.json"
grid_shape_path = "needs_grid_regions_aggregated.geojson"
attach_state_to_buses(base_network, state_shape_path)
attach_grid_region_to_buses(base_network, grid_shape_path)

In [9]:
country_code = "US"
horizon = 2023

three_country_code = convert_two_country_code_to_three(country_code)

In [10]:
base_network.objective

449800296826.11786

---

## 2. Installed capacity 

This section validates the **installed generation capacities** by technology. We compare the capacities in the PyPSA network to those reported by EIA and Ember to ensure the model's infrastructure assumptions are consistent with real-world data.

In [ ]:
installed_capacity_ember = get_installed_capacity_ember(
        ember_data, three_country_code, horizon).round(2)

EIA_inst_capacities = get_data_EIA(
    eia_installed_capacities_path, country_code, horizon)
EIA_inst_capacities = preprocess_eia_data(EIA_inst_capacities).round(2)

installed_capacity_ember.rename({
    "Gas": "Natural gas", "Fossil fuels": "Oil"
}, inplace=True)

In [ ]:
gen_carriers = {
    "onwind", "offwind-ac", "offwind-dc", "solar", "solar rooftop",
    "csp", "nuclear", "geothermal", "ror", "PHS", "hydro", "solid biomass", 
}
link_carriers = {
    "OCGT", "CCGT", "coal", "oil", "biomass", "urban central solid biomass CHP", "gas CHP", "battery discharger"
}

# Generators
gen = base_network.generators.copy()
gen['carrier'] = gen['carrier'].replace({'offwind-ac': 'offwind', 'offwind-dc': 'offwind'})
gen = gen[gen.carrier.isin(gen_carriers)]
gen_totals = gen.groupby('carrier')['p_nom_opt'].sum()

# Storage
sto = base_network.storage_units.copy()
sto = sto[sto.carrier.isin(gen_carriers)]
sto_totals = sto.groupby('carrier')['p_nom_opt'].sum()

# Links (output side scaled by efficiency)
links = base_network.links.copy()
mask = (
    links.efficiency.notnull()
    & (links.p_nom_opt > 0)
    & links.carrier.isin(link_carriers)
)
links = links[mask]
links_totals = links.groupby('carrier').apply(
    lambda df: (df['p_nom_opt'] * df['efficiency']).sum()
)

# Combine all
all_totals = pd.concat([gen_totals, sto_totals, links_totals])
all_totals = all_totals.groupby(all_totals.index).sum()  # Merge duplicates
all_totals = all_totals[all_totals > 0] / 1e3

In [ ]:
pypsa_cap = (
    all_totals.rename({
        "onwind": "Wind", "offwind": "Wind", "solar rooftop": "Solar",
        "solar": "Solar", "ror": "Hydro", "geothermal": "Geothermal", "nuclear": "Nuclear", "hydro": "Hydro",
        "OCGT": "Natural gas", "CCGT": "Natural gas", "oil": "Oil",
        "coal": "Coal", "biomass": "Biomass", "urban central solid biomass CHP": "Biomass", "solid biomass": "Biomass", "battery discharger": "Battery storage"
    })
    .to_frame('PyPSA-Earth results')
    .round(2)
    .groupby(level=0).sum()
)

# skip 0
pypsa_cap = pypsa_cap[pypsa_cap['PyPSA-Earth results'] > 0]

In [ ]:
fossil_fuels = ["Natural gas", "Oil", "Coal"]

# Sum fossil fuels, no KeyError if a carrier is missing
pypsa_fossil_fuels = pypsa_cap.reindex(fossil_fuels, fill_value=0).sum().iloc[0]
ember_fossil_fuels = installed_capacity_ember.reindex(fossil_fuels, fill_value=0).sum().iloc[0]

# Add the aggregated "Fossil fuels" row
pypsa_cap.loc['Fossil fuels'] = pypsa_fossil_fuels
installed_capacity_ember.loc['Fossil fuels'] = ember_fossil_fuels

# Concatenate all dataframes
installed_capacity_df = pd.concat(
    [pypsa_cap, installed_capacity_ember, EIA_inst_capacities], axis=1
).fillna(0)

# Replace 0 with "N/A" for readability
installed_capacity_df_na = installed_capacity_df.replace(0, "N/A")

installed_capacity_df_na


In [ ]:
installed_capacity_df.plot(kind="bar", figsize=(12, 6), width=0.8)
plt.title(f"Installed Capacity by Source in {horizon}")
plt.ylabel("Installed Capacity (GW)")
plt.xticks(rotation=0)
plt.legend(loc='best')
plt.tight_layout()

Installed capacity data source <br>
Ember source: https://ember-energy.org/data/electricity-data-explorer/?data=capacity&entity=United+States <br>
EIA source: https://www.eia.gov/international/data/world/electricity/electricity-capacity

---

## 3. Electricity generation

In this section, we compare the **annual electricity generation** by technology as reported by PyPSA-Earth, EIA, and Ember. This helps to identify any discrepancies in the modeled generation mix and total output.

In [ ]:
# ember_data = load_ember_data()
generation_data_ember = get_generation_capacity_ember_detail(
        ember_data, three_country_code, horizon).round(2)
generation_data_ember.drop(['Load shedding', 'Other Fossil'], inplace=True)

eia_generation = get_data_EIA(eia_generation_path, country_code, horizon)
eia_generation = preprocess_eia_data_detail(eia_generation).round(2)

In [ ]:
pypsa_gen_final = calculate_total_generation_by_carrier(base_network).rename({
        "CCGT": "Natural gas",  "OCGT": "Natural gas", "Csp": "Solar",
        "biomass EOP": "Biomass", "biomass": "Biomass", "urban central solid biomass CHP": "Biomass", "solid biomass": "Biomass",
        "coal": "Coal",         "oil": "Oil", "urban central gas CHP": "Natural gas", "ror": "Hydro", "nuclear": "Nuclear",
        "solar": "Solar", "solar rooftop": "Solar", "hydro": "Hydro", "Reservoir & Dam": "Hydro",
        "onwind": "Wind", "offwind": "Wind", "geothermal": "Geothermal", "battery discharger": "Battery",
    }).to_frame('PyPSA-Earth results').groupby(level=0).sum().round(2)

In [ ]:
generation_df = pd.concat(
    [pypsa_gen_final, generation_data_ember, eia_generation], axis=1).fillna(0)

totals_row = generation_df.sum().to_frame().T
totals_row.index = ['Total']
generation_df_with_totals = pd.concat([generation_df, totals_row])

display(generation_df_with_totals)

In [ ]:
generation_df.plot(kind="bar", figsize=(12, 6), width=0.8)
plt.title(f"Electricity Generation by technology class - {horizon}")
plt.ylabel("Electricity Generation (TWh)")
plt.xticks(rotation=0)
plt.legend(loc='upper right')
plt.tight_layout()

Electricity generation <br>
Ember source: https://ember-energy.org/data/us-electricity-data/ <br>
EIA source: https://www.eia.gov/international/data/world/electricity/electricity-generation

---

## 4. Electricity demand

Finally, we validate the **electricity demand** in the PyPSA network against the EIA and Ember datasets. This ensures that the modeled demand matches observed values for the base year.

### 4.1 Total electricity demand

*Total electricity demand in PyPSA-Earth (US) uses NREL as a reference (see the `scenario_analysis_single` notebook.*

### 4.2 State-wise Total Electricity Demand

*A bar plot showing the state-wise annual electricity demand for the base year (2023) to validate the quality of the spatial distribution of demand.*

In [ ]:
demand_ember = get_demand_ember(ember_data, three_country_code, horizon)
_, pypsa_demand =compute_demand(base_network)

EIA_demand = preprocess_eia_demand(eia_demand_path, horizon)

In [ ]:
base_demand_grid_region, base_demand_state = compute_demand(base_network)

In [ ]:
demand_total =  pd.concat([EIA_demand, base_demand_state.sum(axis=1)], axis=1).rename({0: 'PyPSA-Earth'}, axis=1)

In [ ]:
fig = px.bar(demand_total, barmode='group')
fig.update_layout(width=2500, 
                  height=700, 
                  yaxis_title='Demand (TWh)',
                  xaxis_title='States',
                  title=f'Electricity Demand for {horizon}')
fig.show()

Electricity demand <br>
EIA source: https://www.eia.gov/state/seds/seds-data-complete.php?sid=US#Consumption <br> 
https://www.eia.gov/state/seds/sep_use/total/csv/use_all_phy.xlsx

In [ ]:
total_region_demand = base_demand_grid_region.sum(axis=1)
base_demand_grid_region["Total"] = total_region_demand
base_demand_grid_region.T

In [ ]:
total_region_demand = base_demand_state.sum(axis=1)
base_demand_state["Total"] = total_region_demand
base_demand_state.T

In [ ]:
fig1 = px.bar(base_demand_grid_region.drop(columns=["Total"], errors='ignore'), barmode='stack', text_auto='.1f')
fig1.update_layout(width=1100, yaxis_title='Demand (TWh)', xaxis_title='Grid regions', title='Electricity demand by type of load and Grid region (2023)')
fig1.show()

fig1 = px.bar(base_demand_state.drop(columns=["Total"]), barmode='stack', text_auto='.1f')
fig1.update_layout(width=3000, yaxis_title='Demand (TWh)', xaxis_title='States', title='Electricity Demand by type of load and State (2023)')
fig1.show()